# 如何计算卷积网络的网络参数量

以 `AlexNet` 作为示例网络，输入数据为[1,1,28,28]

In [2]:
import torch
from torch import nn
from d2l import torch as d2l

In [3]:
net = nn.Sequential(
    # 这里使用一个11*11的更大窗口来捕捉对象。
    # 同时，步幅为4，以减少输出的高度和宽度。
    # 另外，输出通道的数目远大于LeNet
    nn.Conv2d(1, 96, kernel_size=11, stride=4, padding=1), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2),
    # 减小卷积窗口，使用填充为2来使得输入与输出的高和宽一致，且增大输出通道数
    nn.Conv2d(96, 256, kernel_size=5, padding=2), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2),
    # 使用三个连续的卷积层和较小的卷积窗口。
    # 除了最后的卷积层，输出通道的数量进一步增加。
    # 在前两个卷积层之后，汇聚层不用于减少输入的高度和宽度
    nn.Conv2d(256, 384, kernel_size=3, padding=1), nn.ReLU(),
    nn.Conv2d(384, 384, kernel_size=3, padding=1), nn.ReLU(),
    nn.Conv2d(384, 256, kernel_size=3, padding=1), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nn.Flatten(),
    # 这里，全连接层的输出数量是LeNet中的好几倍。使用dropout层来减轻过拟合
    nn.Linear(6400, 4096), nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(4096, 4096), nn.ReLU(),
    nn.Dropout(p=0.5),
    # 最后是输出层。由于这里使用Fashion-MNIST，所以用类别数为10，而非论文中的1000
    nn.Linear(4096, 10))

In [4]:
# 计算卷积层的输出大小
def calculate_conv_output_size(input_size, kernel_size, padding, stride):
    return (input_size - kernel_size + 2 * padding) // stride + 1


# 计算汇聚层的输出大小
def calculate_pooling_output_size(input_size, pooling_size, stride):
    return (input_size - pooling_size) // stride + 1

In [5]:
X = torch.randn(1, 1, 224, 224)

total_params = 0

for layer in net:
    if layer.__class__.__name__ == 'MaxPool2d':
        pooling_output_size = calculate_pooling_output_size(X.shape[2], layer.kernel_size, layer.stride)
        print(f"汇聚层输出大小：{pooling_output_size} [核：{layer.kernel_size}，步幅：{layer.stride}]")

    # 卷基层
    if layer.__class__.__name__ == 'Conv2d':
        conv_output_size = calculate_conv_output_size(X.shape[2], layer.kernel_size[0], layer.padding[0],
                                                      layer.stride[0])
        print(f"卷积层输出大小：{conv_output_size} [核：{layer.kernel_size[0]}，"
              f"填充：{layer.padding[0]} 步幅：{layer.stride[0]}]", )

        layer_params = (layer.kernel_size[0] * layer.kernel_size[1] * layer.in_channels + 1) * layer.out_channels
        print(f"卷积层参数大小：{layer_params}")
        total_params += layer_params

    # 全连接层
    if layer.__class__.__name__ == 'Linear':
        layer_params = layer.in_features * layer.out_features
        print(f"全连接层参数大小：{layer_params}")
        total_params += layer_params

    X = layer(X)
    # print(layer.__class__.__name__, 'output shape:\t', X.shape)

print(f"总参数量：{total_params}")

卷积层输出大小：54 [核：11，填充：1 步幅：4]
卷积层参数大小：11712
汇聚层输出大小：26 [核：3，步幅：2]
卷积层输出大小：26 [核：5，填充：2 步幅：1]
卷积层参数大小：614656
汇聚层输出大小：12 [核：3，步幅：2]
卷积层输出大小：12 [核：3，填充：1 步幅：1]
卷积层参数大小：885120
卷积层输出大小：12 [核：3，填充：1 步幅：1]
卷积层参数大小：1327488
卷积层输出大小：12 [核：3，填充：1 步幅：1]
卷积层参数大小：884992
汇聚层输出大小：5 [核：3，步幅：2]
全连接层参数大小：26214400
全连接层参数大小：16777216
全连接层参数大小：40960
总参数量：46756544


In [6]:
# 使用第三方库
from thop import profile

input = torch.randn(1, 1, 224, 224)


def count_your_model(model, x, y):
    print("count")
    # your rule here


macs, params = profile(net, inputs=(input,),
                       custom_ops={net: count_your_model})
print(f'params: {params}')

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
params: 46764746.0


In [9]:
params = sum(p.numel() for p in net.parameters() if p.requires_grad)
print(f'params: {params}')

# 估算显存占用量（仅考虑参数量）
estimated_memory = params * 4  # float32 数据类型每个元素占用4个字节
print(f"Estimated memory usage for parameters: {estimated_memory / 1024 / 1024} MB")

params: 46764746
Estimated memory usage for parameters: 178.39334869384766 MB


**参数量 = 卷积层的参数量 + 全连接层的参数量**

卷积层的参数量 = (卷积核大小 * 卷积核大小 * 输入通道数 + 1) * 输出通道数。
    1 为 bias 数量
全连接层的参数量 = 输入神经元个数 * 输出神经元个数 + B
    B 为 bias 数量

以[1,1,224,224]为输入数据
第一层卷积参数量：(11 * 11 * 1 + 1) * 96
第二层卷积参数量：(5 * 5 * 96 + 1) * 256
...
